In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms

import torchvision
from torch.utils.data import DataLoader
from torch.autograd.variable import Variable

device = torch.device("cuda:0" if torch.cuda.is_available()
                      else "cpu")
torch.manual_seed(1)


In [11]:
DATASET_SCALE = 0.1 # 10% of initial dataset for train, val and test
DATASET_SPLIT = [0.8, 0.2]

# Note to self: moved from /files/ to ./data for lab consistency (see lab 0)


    
temp_set = torchvision.datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST('./data', train=False, download=True, transform=transform)

percentage_split = 0.8
def loader(trainlenth, batch_size):
    train_dataset, val_dataset, _ = torch.utils.data.random_split(temp_set, [int(trainlenth*percentage_split), int(trainlenth-(trainlenth*(percentage_split))), len(temp_set)-trainlenth])
    

    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader, test_loader
#train_dataset, val_dataset = random_split(temp_set, DATASET_SPLIT)





In [26]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_set = torchvision.datasets.MNIST('./data', train=True, download=True, transform=transform)
test_set = torchvision.datasets.MNIST('./data', train=False, download=True, transform=transform)

In [3]:
# From paper
def Std_loss_func(D_real, D_fake, discriminator = True):
     if (discriminator):
          return -torch.mean(torch.log(D_real) + torch.log(1 - D_fake))
     return -torch.mean(torch.log(D_fake))

# task 2
def Logis_loss():
     pass

# Random sample
def sample_Z(m, n=1):
    return torch.randn(m, n)

# Re-implement 
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / (in_dim / 2.)**0.5  # Using standard Python sqrt calculation
    return torch.randn(*size) * xavier_stddev

In [4]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.W1 = nn.Parameter(xavier_init([784, 128]))
        self.b1 = nn.Parameter(torch.zeros(128))
        self.W2 = nn.Parameter(xavier_init([128, 1]))
        self.b2 = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        x = torch.relu(x @ self.W1 + self.b1)
        logits = x @ self.W2 + self.b2
        prob = torch.sigmoid(logits)
        return prob, logits


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.W1 = nn.Parameter(xavier_init([100, 128]))
        self.b1 = nn.Parameter(torch.zeros(128))
        self.W2 = nn.Parameter(xavier_init([128, 784]))
        self.b2 = nn.Parameter(torch.zeros(784))

    def forward(self, z):
        z = torch.relu(z @ self.W1 + self.b1)
        log_prob = z @ self.W2 + self.b2
        prob = torch.sigmoid(log_prob)
        return prob

In [32]:
m = 5
k = 1

def train(loss_criterion=Std_loss_func):
    generator = Generator()
    discriminator = Discriminator()
    D_optimizer = torch.optim.Adam(params=discriminator.parameters(), lr=0.001)
    G_optimizer = torch.optim.Adam(params=generator.parameters(), lr=0.001)
    
    train_loader = DataLoader(train_set, batch_size=1, shuffle=True)

    for it in range(20_000):

        # Assuming train_loader is a DataLoader object for the MNIST dataset.
        print(len(next(iter(train_loader))))
        
        # Update discriminator (k times)
        for jt in range(k):
            for mi in range(m):
                (X_mb_img, x_mb_label)  = next(iter(train_loader))
                X_mb_img = X_mb_img.view(X_mb_img.size(0), -1)  # Flatten the image tensor if needed
                Z = sample_Z(m)  # Sample noise

                D_optimizer.zero_grad()
                D_real, D_logit_real = discriminator(X_mb_img)
                D_fake, D_logit_fake = discriminator(generator(Z)) 

                D_loss_real = loss_criterion(D_logit_real, torch.ones_like(D_real))
                D_loss_fake = loss_criterion(D_logit_fake, torch.zeros_like(D_fake))
                D_loss = (D_loss_real + D_loss_fake) / 2
                D_loss.backward()
                D_optimizer.step()

        # Update generator
        Z = sample_Z(m)  # Sample noise
        G_optimizer.zero_grad()
        G_fake, G_logit_fake = discriminator(generator(Z))
        G_loss = loss_criterion(G_logit_fake, torch.ones_like(G_fake), discriminator=False)
        G_loss.backward()
        G_optimizer.step()

        if it % 10 == 0:
                print(f"Iteration {it}: D_loss {D_loss.item()}, G_loss {G_loss.item()}")

    return discriminator, generator


# testing
D, G = train()

2


RuntimeError: mat1 and mat2 shapes cannot be multiplied (5x1 and 100x128)

In [16]:
# Train with std loss


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
# Train with logistic loss
D, G = train(Logis_loss)